In [ ]:
#sudo /home/henry/.nvm/versions/node/v24.10.0/bin/codex --dangerously-bypass-approvals-and-sandbox

In [6]:
import sys
sys.executable

'/home/henry/projects/navidrome/musicembed/.venv/bin/python'

In [1]:
# Testing batch inference of Music Flamingo
from transformers import (
    AutoModelForCausalLM,
    MusicFlamingoForConditionalGeneration,
    AutoProcessor,
    FineGrainedFP8Config,
    TextStreamer
)
from safetensors.torch import load_file
from huggingface_hub import hf_hub_download
from pathlib import Path
import torch


In [2]:
model = MusicFlamingoForConditionalGeneration.from_pretrained(
    "./music_flamingo_fp8",
    device_map="cuda",
    dtype="auto",
    attn_implementation="flash_attention_2"
)


ImportError: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.

In [3]:
processor = AutoProcessor.from_pretrained("./music_flamingo_fp8")

In [6]:
streamer = TextStreamer(processor)

In [5]:
processor.save_pretrained("music_flamingo_fp8")

['music_flamingo_fp8/processor_config.json']

In [ ]:
torch.set_float32_matmul_precision("high")

model.generation_config.cache_implementation = "static"
model.generation_config.max_new_tokens = 2048
# torch.compile skipped for quantized model


In [7]:

conversation = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Describe this track in full detail - tell me the genre, tempo, and key, then dive into the instruments, production style, and overall mood it creates."},
            {"type": "audio", "path": "https://huggingface.co/datasets/nvidia/AudioSkills/resolve/main/assets/song_1.mp3"},
        ],
    }
]

inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
).to(model.device)

outputs = model.generate(**inputs, streamer=streamer, max_new_tokens=256)

decoded_outputs = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(decoded_outputs[0])

<|im_start|>system
You are Music Flamingo, a multimodal assistant for language and music. On each turn you receive an audio clip which contains music and optional text, you will receive at least one or both; use your world knowledge and reasoning to help the user with any task. Interpret the entirety of the content any input music--regardlenss of whether the user calls it audio, music, or sound.<|im_end|>
<|im_start|>user
<|sound_bos|><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><sound><